In [ ]:
import ktrain
from ktrain import text
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_excel("datas2.xlsx")

# Yeni Bölüm

In [ ]:
review_df = df[['Reviews','sentiment']]

print(review_df.shape)
review_df.head(5)

(13079, 2)


,Reviews,sentiment
0,Very high quality and beautiful Amazon is more...,1
1,The product is really excellent although it re...,1
2,Price is a very good choice in terms of perfor...,1
3,Thanks to Amazon Turkey before the product I g...,1
4,I got it very quickly It was very well preserv...,1


In [ ]:
review_df = review_df[review_df['sentiment'] != 'neutral']
review_df

,Reviews,sentiment
0,Very high quality and beautiful Amazon is more...,1
1,The product is really excellent although it re...,1
2,Price is a very good choice in terms of perfor...,1
3,Thanks to Amazon Turkey before the product I g...,1
4,I got it very quickly It was very well preserv...,1
...,...,...
13074,Good price and quality,1
13075,It is an excellent product It arrived complete...,1
13076,Team fully sealed and in perfect condition rec...,1
13077,I loved it better cost benefit,1


In [ ]:
review_df['sentiment'].value_counts()

1    11615
0      614
Name: sentiment, dtype: int64

In [ ]:
x = df['Reviews'].tolist()

In [ ]:
y = df['sentiment'].tolist()

In [ ]:
X_train, X_val_and_test, y_train, y_val_and_test = train_test_split(x,
                                                                   y,
                                                                   test_size = 0.3)

In [ ]:
X_train, X_val_and_test, y_train, y_val_and_test = train_test_split(x,
                                                                   y,
                                                                   test_size = 0.3)

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_val_and_test,
                                               y_val_and_test,
                                               test_size = 0.3)

In [ ]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_array(x_train = X_train,
                                                                      y_train = y_train,
                                                                      x_test = X_test,
                                                                      y_test = y_test,
                                                                      class_names = ['0', '1'],
                                                                      preprocess_mode = 'bert',
                                                                      ngram_range = 1,
                                                                      maxlen = 512 )

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


/usr/local/lib/python3.8/dist-packages/ktrain/utils.py:745: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  warnings.warn(


task: text classification


In [ ]:
model = text.text_classifier('bert',
                             train_data = (X_train, y_train),
                             preproc = preproc)

Is Multi-Label? False
maxlen is 512
done.


In [ ]:
learner = ktrain.get_learner(model,
                             train_data=(X_train, y_train),
                             batch_size=8)

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input-Token (InputLayer)       [(None, 512)]        0           []                               
                                                                                                  
 Input-Segment (InputLayer)     [(None, 512)]        0           []                               
                                                                                                  
 Embedding-Token (TokenEmbeddin  [(None, 512, 768),  23440896    ['Input-Token[0][0]']            
 g)                              (30522, 768)]                                                    
                                                                                                  
 Embedding-Segment (Embedding)  (None, 512, 768)     1536        ['Input-Segment[0][0]']    

In [ ]:
history = learner.fit_onecycle(1e-5, 1)



begin training using onecycle policy with max lr of 1e-05...
1145/1145 [==============================] - 203s 155ms/step - loss: 0.2609 - accuracy: 0.9152


In [ ]:
learner.validate(val_data=(X_test, y_test))

37/37 [==============================] - 10s 162ms/step
              precision    recall  f1-score   support

           0       0.53      0.63      0.58        54
           1       0.98      0.97      0.97      1043
           2       0.79      0.72      0.75        81

    accuracy                           0.94      1178
   macro avg       0.77      0.77      0.77      1178
weighted avg       0.94      0.94      0.94      1178



array([[  34,   15,    5],
       [  18, 1015,   10],
       [  12,   11,   58]])

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
temp_data = ["the product was simply amazing",
             "the product was extremely terrible"]

In [ ]:
predictor.predict(temp_data)

1/1 [==============================] - 0s 37ms/step


['1', '0']